<a href="https://colab.research.google.com/github/D3Ml4N/Analisis-Exploratorio-de-Datos/blob/main/Automatizacion_taller2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

 # **TALLER No2:** Clasificación.

## *Carga de datos.*

In [2]:
import sqlite3
import numpy as np
import pandas as pd

conexion = sqlite3.connect('datos_sensores.db')

#Código SQL.
query = """
SELECT tamaño, polaris.sensor_polaris, orion.sensor_orion, antares.sensor_antares, vega.sensor_vega, categoria,
CASE
    WHEN clasificacion.etiqueta = "Positivo" THEN 1
	  WHEN clasificacion.etiqueta = "Negativo" THEN 0
END as Clasificacion_bin
FROM datos_basicos
LEFT JOIN clasificacion on datos_basicos.id = clasificacion.id
LEFT JOIN orion on orion.id = datos_basicos.id
LEFT JOIN vega on datos_basicos.id = vega.id
LEFT JOIN polaris on datos_basicos.id = polaris.id
LEFT JOIN antares on datos_basicos.id = antares.id
"""
dataset = pd.read_sql_query(query,conexion)
dataset.head()

,tamaño,sensor_polaris,sensor_orion,sensor_antares,sensor_vega,categoria,Clasificacion_bin
0,0.02,35.36,19.77,-62.73,42.15,3,1.0
1,0.11,69.78,-23.51,-38.02,-20.64,3,1.0
2,0.85,30.82,86.63,-71.62,75.26,1,0.0
3,0.53,71.92,-44.91,27.17,4.11,1,0.0
4,0.14,-2.91,-66.34,-74.93,-48.04,2,0.0


## *Creación y entrenamiento del modelo*

In [32]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
#from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from sklearn.ensemble import RandomForestClassifier
#from sklearn.metrics import accuracy_score
from sklearn.preprocessing import StandardScaler, MinMaxScaler
#import pandas_profiling
#from sklearn.metrics import mean_squared_error

X = dataset.drop('Clasificacion_bin', axis=1)
Y = dataset['Clasificacion_bin']
X.index
T = X.query('index<300000')
L = Y.iloc[:300000]
G = X.query('index>=300000')

T_train,T_test,L_train,L_test = train_test_split(T, L, test_size=0.1)

#Creación del modelo
modelo = RandomForestClassifier(n_estimators=100)
modelo.fit(T_train,L_train)

RandomForestClassifier()

In [33]:
T_test

,tamaño,sensor_polaris,sensor_orion,sensor_antares,sensor_vega,categoria
291616,0.15,62.98,82.45,13.23,92.78,2
177626,0.71,-42.32,69.82,-77.52,-8.84,1
217599,0.99,-62.26,-79.71,-98.12,97.46,2
208157,0.51,-38.88,58.62,-74.10,-3.27,3
60939,0.53,-84.54,98.90,-90.27,6.04,3
...,...,...,...,...,...,...
189942,0.78,3.46,-2.49,-43.28,-51.35,2
277807,0.37,36.41,-47.84,16.14,39.37,1
50506,0.53,18.92,74.50,-46.31,8.97,1
37566,0.07,-82.58,-27.79,7.14,39.86,1


##*Presición del modelo*

In [41]:
#Predicciones
predicciones = modelo.predict(T_test)
predicciones

array([1., 0., 0., ..., 1., 1., 1.])

In [39]:
accuracy = accuracy_score(L_test,predicciones)
print(f"Presición del modelo: {accuracy:.2f}")

Presición del modelo: 0.91


##*Aproximación de los datos faltantes.*

In [42]:
dataset = pd.read_sql_query(query,conexion)
datos_clasificacion = dataset.tail(100)
p = datos_clasificacion.drop('Clasificacion_bin', axis=1)

datos_faltantes = modelo.predict(p)
datos_faltantes

array([0., 0., 1., 1., 1., 1., 0., 0., 1., 1., 1., 0., 1., 1., 1., 1., 1.,
       0., 0., 0., 0., 1., 0., 0., 0., 1., 0., 1., 1., 0., 0., 1., 0., 0.,
       1., 0., 0., 1., 0., 0., 1., 0., 0., 1., 1., 1., 0., 0., 1., 1., 0.,
       0., 1., 0., 1., 1., 1., 0., 1., 1., 0., 0., 1., 1., 0., 0., 1., 1.,
       0., 1., 0., 1., 0., 1., 1., 0., 1., 1., 1., 0., 1., 0., 0., 0., 0.,
       0., 0., 1., 1., 1., 1., 0., 0., 0., 0., 1., 1., 1., 1., 1.])